In [37]:
######### spark session

# Ref : https://github.com/subhamkharwal/pyspark-zero-to-hero/blob/master/05_sort_union_aggregation.ipynb

In [9]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("Spark Session")
         .master("local[*]")
         .getOrCreate()
        )

In [11]:
spark #print session

In [21]:
#Create Dataframe
data = [[1, "ABC", 2000], [2, "DEF", 4000], [3, "GGGG", 3000], [4, 'HHHH', 7000], [5, 'IIIII', 4500]]
df = spark.createDataFrame(data=data, schema="id int, name string, salary int")

In [22]:
df.show() #action

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [23]:
#filter
df.where("salary > 5000").show() 

+---+----+------+
| id|name|salary|
+---+----+------+
|  4|HHHH|  7000|
+---+----+------+



In [25]:
df.rdd.getNumPartitions() # num of partti

8

In [29]:
df.write.format('csv').save("test.csv") #action

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/prashant/pivot/personal/tech/pyspark_samples/test.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [38]:
# Get Active Session and Rename

spark_new = spark.getActiveSession()
spark_new

In [39]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)



In [40]:
df.schema

StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True), StructField('salary', IntegerType(), True)])

In [41]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField('id', IntegerType(), False),
    StructField('name', StringType(), True),
    StructField('salary', IntegerType(), True)
]
)

In [42]:
df1 = spark_new.createDataFrame(data=data, schema=schema)

In [43]:
df1.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [44]:
schema_str = "id int, name string, salary int"
df1 = spark_new.createDataFrame(data=data, schema=schema_str)

In [45]:
df1.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [46]:
from pyspark.sql.functions import expr, col
df1.select(col("id"), df1.name, expr("salary")).show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [48]:
df1.selectExpr("cast(id as int) as id", "name", "salary").show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [52]:
df2 = df1.withColumn("tax", col("salary") * 0.2)

In [54]:
df2.withColumnRenamed("tax", "taxx").show()

+---+-----+------+------+
| id| name|salary|  taxx|
+---+-----+------+------+
|  1|  ABC|  2000| 400.0|
|  2|  DEF|  4000| 800.0|
|  3| GGGG|  3000| 600.0|
|  4| HHHH|  7000|1400.0|
|  5|IIIII|  4500| 900.0|
+---+-----+------+------+



In [56]:
df1.drop("tax").show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  ABC|  2000|
|  2|  DEF|  4000|
|  3| GGGG|  3000|
|  4| HHHH|  7000|
|  5|IIIII|  4500|
+---+-----+------+



In [58]:
from pyspark.sql.functions import lit
columns = {
    "col1" : lit(1),
    "tax" : expr("salary") * 0.2,
    "col2" : lit("col2")
}

In [69]:
df2.withColumns(columns).show()

+---+-----+------+------+----+------+----+
| id| name|salary| bonus|col1|   tax|col2|
+---+-----+------+------+----+------+----+
|  1|  ABC|  2000| 400.0|   1| 400.0|col2|
|  2|  DEF|  4000| 800.0|   1| 800.0|col2|
|  3| GGGG|  3000| 600.0|   1| 600.0|col2|
|  4| HHHH|  7000|1050.0|   1|1400.0|col2|
|  5|IIIII|  4500| 675.0|   1| 900.0|col2|
+---+-----+------+------+----+------+----+



In [70]:
from pyspark.sql.functions import when

In [72]:
df2 = df2.withColumn("bonus", 
               when(col("salary") <= 4000, col("salary") * 0.2)
               .when(col("salary")>4000, col("salary") * 0.15)
               .otherwise(col("salary")*1)
              )

In [73]:
df2.show()

+---+-----+------+------+
| id| name|salary| bonus|
+---+-----+------+------+
|  1|  ABC|  2000| 400.0|
|  2|  DEF|  4000| 800.0|
|  3| GGGG|  3000| 600.0|
|  4| HHHH|  7000|1050.0|
|  5|IIIII|  4500| 675.0|
+---+-----+------+------+



In [78]:
df2.withColumn("tax", expr("case when salary <= 4000 then (salary * 0.1) when salary > 4000 then (salary * 0.2) else 0 end")).show()

+---+-----+------+------+------+
| id| name|salary| bonus|   tax|
+---+-----+------+------+------+
|  1|  ABC|  2000| 400.0| 200.0|
|  2|  DEF|  4000| 800.0| 400.0|
|  3| GGGG|  3000| 600.0| 300.0|
|  4| HHHH|  7000|1050.0|1400.0|
|  5|IIIII|  4500| 675.0| 900.0|
+---+-----+------+------+------+



In [ ]:
# UNION 
'''
Column names, datatypes and sequence of columns of both the dataframes should be same.
union all - keeps duplicate rows
union - keeps distinct records
unionByName - works with diff column seq but number of cols and datatype should be same
'''

In [79]:
# Emp Data & Schema

emp_data_1 = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"]
]

emp_data_2 = [
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [84]:
emp_df1 = spark_new.createDataFrame(data=emp_data_1, schema = emp_schema)
emp_df2 = spark_new.createDataFrame(data=emp_data_2, schema = emp_schema)

In [85]:
emp_df = emp_df1.union(emp_df2)

In [86]:
emp_df.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [90]:
from pyspark.sql.functions import coalesce, lit
emp_df_new = emp_df.withColumn("new_gender", coalesce("gender", lit("O")))

In [91]:
emp_df_new.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|new_gender|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|      Male|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|    Female|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|      Male|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|    Female|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|      Male|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|    Female|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|      Male|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|    Female|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|      Male|
|        010|   

In [94]:
from pyspark.sql.functions import count
emp_df_new.groupBy("department_id").agg(count("employee_id")).alias("employee_count").show()

+-------------+------------------+
|department_id|count(employee_id)|
+-------------+------------------+
|          101|                 3|
|          102|                 4|
|          103|                 4|
|          104|                 3|
|          105|                 2|
|          106|                 2|
|          107|                 2|
+-------------+------------------+

